<a href="https://colab.research.google.com/github/jumpingsphinx/ESE2030/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
def gaussian_elimination(A, b):
    """
    Performs Gaussian elimination on the augmented matrix [A | b] and returns the row operations.
    """
    A = A.astype(float)  # Convert to float for division
    b = b.astype(float)
    n = len(A)
    steps = []

    aug_matrix = np.hstack((A, b.reshape(-1, 1)))
    steps.append(aug_matrix.copy())

    for i in range(n):
        # Pivoting: Swap rows if needed
        max_row = i + np.argmax(np.abs(aug_matrix[i:, i]))
        if max_row != i:
            aug_matrix[[i, max_row]] = aug_matrix[[max_row, i]]
            steps.append(aug_matrix.copy())

        # Normalize pivot row
        aug_matrix[i] = aug_matrix[i] / aug_matrix[i, i]
        steps.append(aug_matrix.copy())

        # Eliminate entries below pivot
        for j in range(i+1, n):
            aug_matrix[j] -= aug_matrix[i] * aug_matrix[j, i]
            steps.append(aug_matrix.copy())

    return steps

def print_elimination_steps(steps):
    """
    Prints the Gaussian elimination steps.
    """
    for i, step in enumerate(steps):
        print(f"Step {i+1}:")
        print(step)
        print("-" * 40)

def get_user_input():
    """
    Gets matrix size and values from user input.
    """
    m = int(input("Enter the number of rows: "))
    n = int(input("Enter the number of columns: "))
    A = np.zeros((m, n))
    b = np.zeros(m)

    print("\nEnter the coefficients of the matrix A row by row:")
    print("(Enter each row as space-separated numbers)")
    for i in range(m):  # Changed from n to m to properly iterate through rows
        row = input(f"Row {i+1}: ").split()
        if len(row) != n:
            raise ValueError(f"Expected {n} values for row {i+1}, but got {len(row)}")
        A[i] = [float(x) for x in row]

    print("\nEnter the values of the solution vector b:")
    print("(Enter space-separated numbers)")
    b_input = input().split()
    if len(b_input) != m:
        raise ValueError(f"Expected {m} values for solution vector, but got {len(b_input)}")
    b = np.array([float(x) for x in b_input])

    return A, b

def print_solutions(final_matrix):
    """
    Prints the solutions from a row-reduced matrix using back substitution.
    """
    n = final_matrix.shape[1] - 1  # Number of variables (matrix columns minus b column)
    solutions = np.zeros(n)

    # Check if system has no solution or infinite solutions
    rank = np.linalg.matrix_rank(final_matrix[:, :-1])
    aug_rank = np.linalg.matrix_rank(final_matrix)

    if rank < aug_rank:
        print("\nThe system has no solution.")
        return
    if rank < n:
        print("\nThe system has infinitely many solutions.")
        return

    # Back substitution
    for i in range(n-1, -1, -1):  # Start from last equation
        solutions[i] = final_matrix[i, -1]  # Start with the constant term
        # Subtract all known terms
        for j in range(i+1, n):
            solutions[i] -= final_matrix[i, j] * solutions[j]
        solutions[i] /= final_matrix[i, i]  # Divide by coefficient of current variable

    print("\nSolutions:")
    for i in range(n):
        print(f"x{i+1} = {solutions[i]:.4f}")

In [ ]:
# Usage
A, b = get_user_input()
steps = gaussian_elimination(A, b)
print_elimination_steps(steps)
print_solutions(steps[-1])